In [2]:
import pandas as pd
import copy
import pdb
import numpy as np
import copy as cp
from scipy.optimize import minimize
import shutil
import os
import csv
import matplotlib.pyplot as plt

# Step 1. Import data 
Note: the imported data is processed in Stata before importing here. 
Three decision rules: 

******Rule 1: Optimal updating (understand the dynamics + responsive to the waiting cost)

*****"Rule 1 relaxed on the waiting cost part, fully understand the network dynamics"

******Rule 2: Impatient subjects (always 0)

******Rule 3: Patient subjects (understanding the dynamics)


In [2]:
datafolder = "C:\\Users\\xinxi\\Purdue\\EffectiveSocialLearningProject\\DataAnalysis\\ImportedData\\"

In [3]:
data = pd.read_csv(datafolder+"decision_rules"+".csv", sep=",")
data

,subject_id,session,network_type,waiting_cost,round_id,p_node,g_match,within_group_round,RULE1_relaxed,RULE1,RULE2,RULE3
0,49,4,a,H,23,B,8,2,1,1,0,0
1,33,3,a,L,10,B,4,1,0,0,1,0
2,49,4,a,H,11,D,4,2,1,1,1,0
3,35,3,a,L,26,B,9,2,1,1,0,1
4,58,5,a,H,11,E,4,2,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...
2995,88,8,b,L,19,C,7,1,0,0,1,0
2996,62,6,b,H,25,D,9,1,1,1,1,0
2997,99,9,b,L,30,B,10,0,0,0,1,0
2998,65,6,b,H,23,A,8,2,1,1,0,0


In [4]:
data.subject_id.unique()
gb = data.groupby(["subject_id", "p_node"])[["RULE1", "RULE1_relaxed","RULE2","RULE3"]].sum()
# gb = data.groupby(["subject_id"]).aggregate({'RULE1' : 'sum', "RULE1_relaxed": 'sum',"RULE2": 'sum',"RULE3": 'sum'})
gb

RULE1  RULE1_relaxed  RULE2  RULE3
subject_id p_node                                    
11         A           3              3      3      3
           B           0              0      0      0
           C           2              2      0      2
           D           5              5      0      5
           E           0              0      0      6
...                  ...            ...    ...    ...
1010       A           0              0      4      0
           B           1              1      2      0
           C           1              1      3      0
           D           2              2      5      1
           E           1              1      3      1

[500 rows x 4 columns]

In [5]:
data.subject_id.unique()
gb = data.groupby(["subject_id","g_match"])[["RULE1", "RULE1_relaxed","RULE2","RULE3"]].sum()
# gb = data.groupby(["subject_id"]).aggregate({'RULE1' : 'sum', "RULE1_relaxed": 'sum',"RULE2": 'sum',"RULE3": 'sum'})
print(gb.loc)

In [18]:
datafolder = "C:\\Users\\xinxi\\Purdue\\EffectiveSocialLearningProject\\DataAnalysis\\ImportedData\\"
data = pd.read_csv(datafolder+"Networkb_CostH"+".csv", sep=",")
Decide_0 = data.iloc[:,3:].to_numpy()==0
Decide_0*1

array([[1, 0, 1, 1, 1],
       [1, 0, 1, 0, 0],
       [1, 0, 0, 0, 0],
       [0, 1, 0, 0, 0],
       [0, 0, 0, 0, 1],
       [0, 0, 1, 0, 0],
       [1, 1, 0, 1, 1],
       [0, 0, 0, 0, 0],
       [0, 1, 0, 0, 0],
       [0, 0, 0, 1, 0],
       [1, 0, 1, 0, 0],
       [0, 1, 0, 1, 1],
       [0, 0, 0, 0, 0],
       [0, 0, 0, 1, 0],
       [0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0],
       [1, 0, 1, 0, 1],
       [1, 0, 0, 1, 0],
       [0, 0, 0, 1, 0],
       [0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0],
       [0, 1, 0, 1, 1],
       [0, 0, 0, 0, 1],
       [0, 0, 1, 0, 0],
       [1, 0, 0, 0, 0],
       [0, 0, 0, 0, 0],
       [0, 0, 0, 0, 1],
       [0, 1, 0, 0, 0],
       [0, 0, 0, 1, 0],
       [0, 0, 0, 1, 0],
       [0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0],
       [1, 0, 0, 0, 0],
       [0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0],
       [0, 0, 0, 1, 0],
       [0, 0, 0, 0, 0],
       [1, 0, 0, 1, 0],
       [0, 0, 1,

In [19]:
 np.dot(Decide_0,np.array([1,1,1,1,1]))

array([4, 2, 1, 1, 1, 1, 4, 0, 1, 1, 2, 3, 0, 1, 0, 0, 0, 3, 2, 1, 0, 0,
       0, 3, 1, 1, 1, 0, 1, 1, 1, 1, 0, 0, 1, 0, 0, 0, 1, 0, 2, 1, 1, 0,
       2, 1, 1, 2, 2, 1, 2, 4, 1, 3, 0, 2, 3, 0, 1, 0, 1, 1, 1, 0, 3, 0,
       1, 0, 1, 0, 1, 0, 0, 1, 4, 2, 1, 1, 0, 0, 2, 0, 0, 2, 0, 1, 1, 0,
       1, 0, 0, 1, 2, 0, 1, 0, 1, 0, 2, 1, 0, 2, 0, 2, 2, 0, 0, 1, 2, 0,
       2, 1, 1, 0, 2, 1, 1, 1, 1, 1])

# Step 2. Define Likelihood function /

### Define an error rate that is the same across all subjects
Formulating the likelihood function

You need to write down the Likelihood function that takes as an input a vector of proportions of strategies and returns the likelihood value(hw5_q1_final(basic))

In [44]:
def likelihood(x):
    '''x is a len(availabel strategies)+1 vector, first is the beta/gamma, unknown errors same for all strategies'''
    #YOUR CODE GOES HERE
    likelihood = 0 
    
    decision_rules = ["RULE1","RULE2","RULE3"] 
    gb = data.groupby(["subject_id"])[decision_rules].sum()
    gb_size = data.groupby(["subject_id"])[decision_rules].count()

    sub_likelihood = []
    for subject_id in data.subject_id.unique() :
        p_s = 0
        subject_prob = []
        
        for i in range(len(decision_rules)):
            # Total number of rounds meets the decision rule
            correct = gb[decision_rules[i]].loc[subject_id]
            N = gb_size[decision_rules[i]].loc[subject_id]
            wrong = N-correct
            
            pi_s = x[i+1] * np.power(x[0], correct) * np.power((1-x[0]), wrong)       
            p_s += pi_s
#             print(str(stra)+str(i))
            ln_ps = np.log(p_s)
#         print(str(ln_ps))
        likelihood += ln_ps
                      
    return -likelihood

def constraint1(x): #x[0] - gamma, x[1:] - proportions of all strategies
    
    return x[1:].sum()-1

In [45]:
x0 = np.zeros(4)
x0[0] = np.random.random()
temp = np.random.random(3) 
temp = temp/temp.sum()
x0[1:]=temp
x0

array([0.756, 0.320, 0.397, 0.283])

In [46]:

likelihood(x0)

2287.113102172984

#### Optimization

You need to find the parameter that minimizes the -likelihood. Hint: think about potential problems associated with this optimization

In [47]:
b = (0.0,1) # indicates bounds here, since it's probability, so between 0 and 1 
beta = (0.6,1)
#bnds = (b,b,b,b,b,b,b)
bnds = (beta,b,b,b)

simulaiton = 1
currentmin = 0
current_solution = []
while simulaiton <=50:
    x0 = np.zeros(4)
    #Some random starting point
    x0[0] = np.random.random()
    temp = np.random.random(3) #six proportions
    temp = temp/temp.sum()
    x0[1:]=temp

    con1 = {'type': 'eq', 'fun': constraint1} 
    cons = ([con1])

    solution = minimize(likelihood,x0,method='SLSQP',bounds=bnds,constraints=cons)
    if currentmin > likelihood(solution.x):
        currentmin = likelihood(solution.x)
        current_solution = solution.x

    simulaiton +=1
    print(simulaiton)

2
3


<ipython-input-44-d2d67f4d64f3>:24: RuntimeWarning: divide by zero encountered in log
  ln_ps = np.log(p_s)


4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51


In [ ]:
solution.x

In [ ]:
likelihood(solution.x)# when prob = 0.5; 2079.441541679832
# when prob = 0.6, 2093



In [ ]:
data.p_node.unique()[0]

### Define the same error rate but allow different strategies across subjects

In [206]:
def likelihood_Serror_Dstrategy(x_input):
    '''x is a (len(availabel strategies)-2) * # subjects +1 vector,
    first is the beta/gamma, unknown errors same for all strategies'''

    
    #YOUR CODE GOES HERE
    likelihood = 0 
    
    decision_rules = ["RULE1","RULE2","RULE3"] 
    gb = data.groupby(["subject_id"])[decision_rules].sum()
    gb_size = data.groupby(["subject_id"])[decision_rules].count()
    
    N_sub = len(data.subject_id.unique())
    N_decision = len(decision_rules)
    
    x_weight = x_input[1::]
    x_weight = np.reshape(x_weight, (-1,N_decision))
    x_error = x_input[0]
    

    sub_likelihood = []
    j = 0
    for subject_id in data.subject_id.unique() :
        p_s = 0
        subject_prob = []

        for i in range(N_decision):
            # Total number of rounds meets the decision rule
            correct = gb[decision_rules[i]].loc[subject_id]
            N = gb_size[decision_rules[i]].loc[subject_id]
            wrong = N-correct
            
            pi_s = x_weight[j,i] * np.power(x_error, correct) * np.power((1-x_error), wrong)       
            p_s += pi_s
#             print(str(stra)+str(i))
            ln_ps = np.log(p_s)
               
#         print(str(ln_ps))
        likelihood += ln_ps
        j += 1
                      
    return -likelihood

# def constraint1(x): #x[0] - gamma, x[1:] - proportions of all strategies
    
#     return x[1:].sum()-1

In [212]:
b = (0.0,1) # indicates bounds here, since it's probability, so between 0 and 1 
beta = (0.6,1)
#bnds = (b,b,b,b,b,b,b)
bnds_sub = [b,b,b] * N_sub
bnds = tuple([beta]) + tuple(bnds_sub)


decision_rules = ["RULE1","RULE2","RULE3"] 
N_sub = len(data.subject_id.unique())
N_decision = len(decision_rules)


x0 = np.zeros(1+ N_decision* N_sub)
x0[0]= np.random.random()
temp = np.random.random(N_decision*N_sub)
temp = np.reshape(temp,(-1, N_decision))
temp_div = temp.sum(axis = 1).reshape(N_sub, 1)
temp_f = temp / temp_div
x0[1::] = temp_f.flatten()



from functools import partial

def f_constraint(x, index):
    return  np.reshape(x[1::], (-1, N_decision))[index,0::].sum() - 1

cons=[]
for i in range(N_sub):
    cons.append({'type': 'eq', 'fun':  partial(f_constraint, index=i) })

minimize(likelihood_Serror_Dstrategy,x0,method='SLSQP',bounds=bnds,constraints=cons)

<ipython-input-206-e8dd7fc2006a>:36: RuntimeWarning: divide by zero encountered in log
  ln_ps = np.log(p_s)


     fun: 2013.6464504512107
     jac: array([0.135, -1.000, -0.006, -0.000, -0.004, -1.000, 0.000, -0.281,
       -0.004, -1.000, -1.000, -0.022, -0.006, -0.022, -0.004, -1.000,
       -1.000, -0.014, -1.000, -1.000, -0.079, -0.000, -0.006, 0.000,
       -1.000, -1.000, -0.079, -0.079, -0.079, -0.004, -1.000, -0.184,
       -0.001, -1.000, -1.000, -0.022, -0.429, -1.000, -0.034, -0.655,
       -1.000, -0.009, -0.079, -1.000, -0.120, -0.002, -1.000, -0.281,
       -0.281, -0.655, -1.000, -0.079, -0.022, -0.000, -1.000, -1.000,
       -0.184, -0.003, -1.000, -0.079, -0.655, -1.000, -0.014, -0.034,
       -0.079, -0.000, -1.000, -1.000, -0.006, -0.000, -1.000, -0.006,
       -0.009, -1.000, -0.006, -1.000, -1.000, -0.003, -0.001, -0.079,
       -1.000, -0.001, -0.006, -1.000, 0.000, -1.000, -0.006, 0.000,
       -0.022, -0.000, -1.000, -0.002, -0.000, -1.000, -1.000, -0.034,
       -0.429, -0.655, -0.034, -1.000, -1.000, -0.014, -0.001, -0.006,
       -1.000, 0.000, -0.429, -0.009, -1.00

In [218]:
likelihood_Serror_Dstrategy(solution.x)

2132.7596293233137

In [222]:
b = (0.0,1) # indicates bounds here, since it's probability, so between 0 and 1 
beta = (0.6,1)
#bnds = (b,b,b,b,b,b,b)
bnds_sub = [b,b,b] * N_sub
bnds = tuple([beta]) + tuple(bnds_sub)


decision_rules = ["RULE1","RULE2","RULE3"] 
N_sub = len(data.subject_id.unique())
N_decision = len(decision_rules)

simulaiton = 1
currentmin = 0
current_solution = []
while simulaiton <=10:
    

    x0 = np.zeros(1+ N_decision* N_sub)
    x0[0]= np.random.random()
    temp = np.random.random(N_decision*N_sub)
    temp = np.reshape(temp,(-1, N_decision))
    temp_div = temp.sum(axis = 1).reshape(N_sub, 1)
    temp_f = temp / temp_div
    x0[1::] = temp_f.flatten()



    from functools import partial

    def f_constraint(x, index):
        return  np.reshape(x[1::], (-1, N_decision))[index,0::].sum() - 1

    cons=[]
    for i in range(N_sub):
        cons.append({'type': 'eq', 'fun':  partial(f_constraint, index=i) })

  

    solution = minimize(likelihood_Serror_Dstrategy,x0,method='SLSQP',bounds=bnds,constraints=cons)
    if currentmin > likelihood(solution.x):
        currentmin = likelihood(solution.x)
        current_solution = solution.x

    simulaiton +=1
    print(simulaiton)

<ipython-input-206-e8dd7fc2006a>:36: RuntimeWarning: divide by zero encountered in log
  ln_ps = np.log(p_s)


2
3
4
5
6
7
8
9
10
11


In [233]:
likelihood_Serror_Dstrategy(solution.x)
result = solution.x[1::]
result = np.reshape(result, (-1,3))
df = pd.DataFrame(result)
result

array([[1.000, 0.000, 0.000],
       [0.000, 1.000, 0.000],
       [0.000, 0.000, 1.000],
       [1.000, 0.000, 0.000],
       [0.000, 0.000, 1.000],
       [0.383, 0.000, 0.617],
       [1.000, 0.000, 0.000],
       [0.000, 0.000, 1.000],
       [1.000, 0.000, 0.000],
       [0.000, 0.000, 1.000],
       [0.000, 0.000, 1.000],
       [1.000, 0.000, 0.000],
       [1.000, 0.000, 0.000],
       [1.000, 0.000, 0.000],
       [1.000, 0.000, 0.000],
       [1.000, 0.000, 0.000],
       [0.000, 1.000, 0.000],
       [0.000, 0.000, 1.000],
       [1.000, 0.000, 0.000],
       [1.000, 0.000, 0.000],
       [1.000, 0.000, 0.000],
       [0.000, 0.000, 1.000],
       [1.000, 0.000, 0.000],
       [1.000, 0.000, 0.000],
       [0.594, 0.000, 0.406],
       [1.000, 0.000, 0.000],
       [0.000, 1.000, 0.000],
       [0.000, 1.000, 0.000],
       [1.000, 0.000, 0.000],
       [0.000, 0.000, 1.000],
       [0.000, 0.000, 1.000],
       [1.000, 0.000, 0.000],
       [0.000, 0.000, 1.000],
       [1.

### Define the same weight on different strategies but allow different error rates across subjects

In [234]:
def likelihood_Derror_Sstrategy(x_input):
    '''x is a len(availabel strategies) +1 * # subjects vector, first len(available strategies) are the weights
        The rest is the beta/gamma, unknown errors, different for each subject'''
    
    #YOUR CODE GOES HERE
    likelihood = 0 
    
    decision_rules = ["RULE1","RULE2","RULE3"] 
    gb = data.groupby(["subject_id"])[decision_rules].sum()
    gb_size = data.groupby(["subject_id"])[decision_rules].count()
    
    x_weight = x_input[0:len(decision_rules)]
    
    x_error = x_input[len(decision_rules)::]
   

    sub_likelihood = []
    j = 0
    for subject_id in data.subject_id.unique() :
        p_s = 0
        subject_prob = []
        
        for i in range(len(decision_rules)):
            # Total number of rounds meets the decision rule
            correct = gb[decision_rules[i]].loc[subject_id]
            N = gb_size[decision_rules[i]].loc[subject_id]
            wrong = N-correct
    
            pi_s = x_weight[i] * np.power(x_error[j], correct) * np.power((1-x_error[j]), wrong)       
            p_s += pi_s
#             print(str(stra)+str(i))
            ln_ps = np.log(p_s)
#         print(str(ln_ps))
        likelihood += ln_ps
        j += 1
                      
    return -likelihood

def constraint1(x_input): #x[0] - gamma, x[1:] - proportions of all strategies
    decision_rules = ["RULE1","RULE2","RULE3"] 
    
    x_weight = x_input[0:len(decision_rules)]
    
    
    return x_weight[0:].sum()-1

#### Optimization 

In [235]:
b = (0.0,1) # indicates bounds here, since it's probability, so between 0 and 1 
beta = (0.5,1)
#bnds = (b,b,b,b,b,b,b)
bnds_sub = [beta] * N_sub
bnds = (b,b,b) + tuple(bnds_sub)

decision_rules = ["RULE1","RULE2","RULE3"] 
N_sub = len(data.subject_id.unique())
N_decision = len(decision_rules)

simulaiton = 1
currentmin = 0
current_solution = []
while simulaiton <=50:
    
    x0 = np.zeros(N_decision+N_sub)
    temp = np.random.random(N_decision) #six proportions
    temp = temp/temp.sum()
    x0[0:N_decision]=temp
    x0[N_decision::] = np.random.random(N_sub)
    
    con1 = {'type': 'eq', 'fun': constraint1} 
    cons = ([con1])

    solution = minimize(likelihood_Derror_Sstrategy,x0,method='SLSQP',bounds=bnds,constraints=cons)
    if currentmin > likelihood(solution.x):
        currentmin = likelihood(solution.x)
        current_solution = solution.x

    simulaiton +=1
    print(simulaiton)

<ipython-input-234-22eddd3fbc03>:32: RuntimeWarning: divide by zero encountered in log
  ln_ps = np.log(p_s)


2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19


KeyboardInterrupt: 

In [239]:
solution.x
# error_rate = np.reshape(solution.x[3::], (-1,1))
# error_rate

array([0.687, 0.263, 0.050, 0.900, 0.933, 0.500, 0.632, 0.500, 0.500,
       0.800, 0.933, 0.500, 0.500, 0.500, 0.500, 0.500, 0.500, 0.629,
       0.500, 0.500, 0.500, 0.732, 0.500, 0.500, 0.799, 0.867, 0.598,
       0.500, 0.767, 0.500, 1.000, 0.967, 0.500, 0.500, 0.500, 0.500,
       0.700, 1.000, 0.500, 0.933, 0.500, 0.500, 0.699, 0.514, 0.551,
       0.833, 0.600, 0.500, 0.900, 0.500, 1.000, 1.000, 0.500, 0.500,
       0.500, 0.500, 0.500, 0.733, 0.500, 0.500, 0.733, 0.500, 0.500,
       0.500, 0.833, 0.500, 0.699, 0.833, 0.867, 0.500, 0.500, 0.500,
       0.500, 0.500, 0.633, 0.500, 0.500, 0.500, 0.500, 0.733, 0.500,
       1.000, 0.500, 0.500, 0.500, 0.700, 0.500, 0.700, 0.500, 0.933,
       0.500, 0.800, 0.500, 0.500, 0.500, 0.500, 1.000, 0.500, 0.633,
       1.000, 0.500, 0.667, 0.500])

In [124]:
solution.x
[When simulation lasts for 50 times!]
0.051, 0.000, 0.949, 

Subjects prob: 
       1.000, 0.831, 0.789, 0.500, 0.819, 0.500,
       0.784, 0.500, 0.500, 0.500, 0.500, 0.849, 0.500, 0.683, 0.994,
       0.856, 0.822, 0.938, 0.500, 0.500, 0.977, 0.500, 0.500, 0.560,
       0.500, 0.500, 0.500, 0.787, 0.500, 0.500, 0.500, 0.500, 0.500,
       0.500, 0.834, 0.500, 0.656, 0.500, 0.500, 0.957, 0.933, 0.500,
       0.500, 0.500, 0.720, 0.752, 0.619, 0.633, 0.500, 0.500, 0.500,
       0.626, 0.500, 0.504, 0.500, 0.897, 0.720, 0.792, 0.597, 0.500,
       0.597, 0.946, 0.840, 0.740, 0.500, 0.500, 0.967, 0.500, 0.957,
       0.500, 0.500, 0.530, 0.774, 0.500, 0.791, 0.770, 0.500, 0.771,
       0.912, 0.652, 0.500, 0.745, 0.500, 0.593, 0.752, 0.861, 0.500,
       0.500, 0.500, 0.515, 0.500, 0.673, 0.727, 0.612, 0.500, 0.500,
       0.641, 0.764, 0.500, 0.940]

array([0.051, 0.000, 0.949, 1.000, 0.831, 0.789, 0.500, 0.819, 0.500,
       0.784, 0.500, 0.500, 0.500, 0.500, 0.849, 0.500, 0.683, 0.994,
       0.856, 0.822, 0.938, 0.500, 0.500, 0.977, 0.500, 0.500, 0.560,
       0.500, 0.500, 0.500, 0.787, 0.500, 0.500, 0.500, 0.500, 0.500,
       0.500, 0.834, 0.500, 0.656, 0.500, 0.500, 0.957, 0.933, 0.500,
       0.500, 0.500, 0.720, 0.752, 0.619, 0.633, 0.500, 0.500, 0.500,
       0.626, 0.500, 0.504, 0.500, 0.897, 0.720, 0.792, 0.597, 0.500,
       0.597, 0.946, 0.840, 0.740, 0.500, 0.500, 0.967, 0.500, 0.957,
       0.500, 0.500, 0.530, 0.774, 0.500, 0.791, 0.770, 0.500, 0.771,
       0.912, 0.652, 0.500, 0.745, 0.500, 0.593, 0.752, 0.861, 0.500,
       0.500, 0.500, 0.515, 0.500, 0.673, 0.727, 0.612, 0.500, 0.500,
       0.641, 0.764, 0.500, 0.940])

solution.x [FOr 5 simulations!!!]
#array([0.051, 0.000, 0.949, 
(large weight on the "patient subject")
Prob: (high heterogeneity):
        1.000, 0.988, 0.500, 0.628, 0.500, 0.532,
       0.654, 0.688, 0.500, 0.500, 0.623, 0.500, 0.500, 0.744, 0.664,
       0.500, 0.655, 0.500, 0.500, 0.500, 0.500, 0.688, 0.500, 0.500,
       0.500, 0.500, 0.500, 0.767, 0.500, 0.500, 0.500, 0.979, 0.983,
       0.700, 0.500, 0.805, 0.935, 0.500, 0.500, 0.500, 0.883, 0.610,
       0.500, 0.866, 0.842, 0.935, 0.651, 0.897, 0.500, 0.500, 0.500,
       0.518, 0.580, 0.670, 0.500, 0.785, 0.500, 0.500, 0.500, 0.749,
       0.500, 0.500, 0.577, 0.994, 0.694, 0.500, 0.862, 0.669, 0.500,
       0.500, 0.500, 0.728, 0.500, 0.500, 0.640, 0.500, 0.847, 0.500,
       0.500, 0.908, 0.999, 0.500, 0.752, 0.506, 0.500, 0.719, 0.500,
       0.526, 0.500, 0.755, 0.500, 0.828, 0.500, 0.948, 0.990, 0.655,
       0.804, 0.908, 0.500, 0.500])

### Define an error rate that is the same across different positions

In [138]:
def likelihood_node(x):
    '''x is a len(p_node) [5] * len(availabel strategies)+1  vector, 
    first is the beta/gamma, unknown errors same for all strategies
    '''
    x = np.reshape(x, (-1,4))
    
    #YOUR CODE GOES HERE
    likelihood = 0 
    
    decision_rules = ["RULE1","RULE2","RULE3"] 
    gb = data.groupby(["subject_id","p_node"])[decision_rules].sum()
    gb_size = data.groupby(["subject_id","p_node"])[decision_rules].count()
    node_list = data.p_node.unique()
#     print(node_list)

    sub_likelihood = []
    for subject_id in data.subject_id.unique() :
        p_s = 0
        subject_prob = []
        for j in range(len(node_list)) :
            for i in range(len(decision_rules)):
                # Total number of rounds meets the decision rule
                correct = gb[decision_rules[i]].loc[subject_id, node_list[j] ]
                N = gb_size[decision_rules[i]].loc[subject_id,  node_list[j] ]
                wrong = N-correct

                pi_s = x[j, i+1] * np.power(x[j, 0], correct) * np.power((1-x[j, 0]), wrong)       
                p_s += pi_s
    #             print(str(stra)+str(i))
                ln_ps = np.log(p_s)
    #         print(str(ln_ps))
            likelihood += ln_ps

    return -likelihood

# def constraint1_node(x): #x[0] - gamma, x[1:] - proportions of all strategies
#     x = np.reshape(x, (-1,4))
#     return sum(x[:,1:].sum(axis=1) - np.ones(len(x)))
def constraint1_node(x): #x[0] - gamma, x[1:] - proportions of all strategies
    x = np.reshape(x, (-1,4))
    return x[1,1:].sum() - 1
def constraint2_node(x): #x[0] - gamma, x[1:] - proportions of all strategies
    x = np.reshape(x, (-1,4))
    return x[2,1:].sum() - 1
def constraint3_node(x): #x[0] - gamma, x[1:] - proportions of all strategies
    x = np.reshape(x, (-1,4))
    return x[3,1:].sum() - 1
def constraint4_node(x): #x[0] - gamma, x[1:] - proportions of all strategies
    x = np.reshape(x, (-1,4))
    return x[4,1:].sum() - 1
def constraint0_node(x): #x[0] - gamma, x[1:] - proportions of all strategies
    x = np.reshape(x, (-1,4))
    return x[0,1:].sum() - 1

def constraint_node(x): #x[0] - gamma, x[1:] - proportions of all strategies
    x = np.reshape(x, (-1,4))
    return x

for i in range(5):
    name = constraint+str(i)+test
    def name(x) :
        x = np.repshape(x, (-1,4))
        return print(i)

NameError: name 'constraint' is not defined

In [126]:
x0 = np.zeros((5,4))
for i in range(5):
    x0[i, 0] = np.random.random()
    temp = np.random.random(3) 
    temp = temp/temp.sum()
    x0[i, 1:]=temp
x0


array([[0.872, 0.165, 0.393, 0.442],
       [0.348, 0.468, 0.065, 0.467],
       [0.190, 0.180, 0.423, 0.397],
       [0.820, 0.350, 0.287, 0.363],
       [0.380, 0.469, 0.346, 0.185]])

In [127]:
likelihood_node(x0)

1278.2885493833448

In [ ]:
eq_cons = {'type': 'eq',
           'fun' : lambda x: np.array([2*x[0] + x[1] - 1]),
           'jac' : lambda x: np.array([2.0, 1.0])}

In [137]:
x0

array([[0.872, 0.165, 0.393, 0.442],
       [0.348, 0.468, 0.065, 0.467],
       [0.190, 0.180, 0.423, 0.397],
       [0.820, 0.350, 0.287, 0.363],
       [0.380, 0.469, 0.346, 0.185]])

In [130]:
cons = []
for i in range(5) :
    con =  {'type': 'eq', 'fun': lambda x : np.reshape(x, (-1, 4))[i,1::].sum() - 1} 
    cons.append(con)
print(cons)

[{'type': 'eq', 'fun': <function <lambda> at 0x000001EA0EB85670>}, {'type': 'eq', 'fun': <function <lambda> at 0x000001EA0EB854C0>}, {'type': 'eq', 'fun': <function <lambda> at 0x000001EA0EB853A0>}, {'type': 'eq', 'fun': <function <lambda> at 0x000001EA0EB855E0>}, {'type': 'eq', 'fun': <function <lambda> at 0x000001EA0EB85E50>}]


In [143]:
b = (0.0,1) # indicates bounds here, since it's probability, so between 0 and 1 
beta = (0.5,1)
#bnds = (b,b,b,b,b,b,b)
bnds_row = (beta,b,b,b)
bnds = bnds_row *5

x00 = x0.flatten()
# con0 =  {'type': 'eq', 'fun': constraint0_node} 
# con1 =  {'type': 'eq', 'fun': constraint1_node} 
# con2 =  {'type': 'eq', 'fun': constraint2_node} 
# con3 =  {'type': 'eq', 'fun': constraint3_node} 
# con4 =  {'type': 'eq', 'fun': constraint4_node} 
# cons = ([con0,con1,con2,con3,con4])
# cons = ([{'type': 'eq', 'fun': lambda x : np.reshape(x, (-1, 4))[0,1::].sum() - 1},
#         {'type': 'eq', 'fun': lambda x : np.reshape(x, (-1, 4))[1,1::].sum() - 1},
#         {'type': 'eq', 'fun': lambda x : np.reshape(x, (-1, 4))[2,1::].sum() - 1},
#         {'type': 'eq', 'fun': lambda x : np.reshape(x, (-1, 4))[3,1::].sum() - 1},
#         {'type': 'eq', 'fun': lambda x : np.reshape(x, (-1, 4))[4,1::].sum() - 1}])
# cons = []
# for i in range(5) :
#     con =  {'type': 'eq', 'fun':  lambda x : np.reshape(x, (-1, 4))[i,1::].sum() - 1} 
#     cons.append(con)

from functools import partial

def f_constraint(x, index):
    return  np.reshape(x, (-1, 4))[index,1::].sum() - 1

cons=[]
for i in range(5):
    cons.append({'type': 'eq', 'fun':  partial(f_constraint, index=i) })
# print(cons)
minimize(likelihood_node,x00,method='SLSQP',bounds=bnds,constraints=cons)

<ipython-input-138-fc37a3d27ead>:30: RuntimeWarning: divide by zero encountered in log
  ln_ps = np.log(p_s)


     fun: 1417.9323319069297
     jac: array([-0.002, -245.852, -245.860, -245.852, 574.594, -121.734, -121.734,
       -121.734, 284.292, -69.831, -69.831, -69.831, -101.050, -45.709,
       -45.713, -45.714, 42.724, -16.871, -16.871, -16.871])
 message: 'Optimization terminated successfully'
    nfev: 547
     nit: 26
    njev: 25
  status: 0
 success: True
       x: array([0.837, 0.791, 0.166, 0.043, 0.500, 0.486, 0.047, 0.467, 0.500,
       0.171, 0.414, 0.416, 1.000, 0.746, 0.233, 0.022, 0.500, 0.469,
       0.346, 0.185])

In [ ]:
Wrong index?
fun: 1632.1807562772549
     jac: array([377.122, -798.699, -143.509, -100.411, 629.362, -131.536, -131.536,
       -131.536, 323.189, -74.436, -74.436, -74.436, -208.032, -81.925,
       -27.089, -19.344, 43.111, -18.134, -18.134, -18.134])
 message: 'Singular matrix C in LSQ subproblem'
    nfev: 21
     nit: 1
    njev: 1
  status: 6
 success: False
       x: array([0.872, 0.165, 0.393, 0.442, 0.500, 0.468, 0.065, 0.467, 0.500,
       0.180, 0.423, 0.397, 0.820, 0.350, 0.287, 0.363, 0.500, 0.469,
       0.346, 0.185])

In [32]:
print(data.p_node.unique())

*** first time:
     fun: 1455.4634189207438
     jac: array([1116.690, -224.159, -224.159, -224.159, 654.525, -124.159,
       -124.159, -124.159, 251.698, -74.159, -74.159, -74.159, -19.532,
       -44.771, -44.765, -44.769, -61.878, -32.756, -32.755, -32.754])
 message: 'Optimization terminated successfully'
    nfev: 261
     nit: 12
    njev: 12
  status: 0
 success: True
       x: array([0.500, 0.369, 0.390, 0.241, 0.500, 0.281, 0.364, 0.355, 0.500,
       0.704, 0.033, 0.263, 1.000, 0.763, 0.200, 0.037, 1.000, 0.676,
       0.067, 0.257])
**** second time:
        fun: 1343.8430812538954
     jac: array([1649.901, -249.142, -249.142, -249.142, 0.000, -103.489, -103.485,
       -85.556, 447.419, -82.679, -82.679, -82.679, 0.019, -35.582,
       -35.572, -35.609, -17.633, -29.112, -29.121, -29.107])
 message: 'Optimization terminated successfully'
    nfev: 394
     nit: 22
    njev: 18
  status: 0
 success: True
       x: array([0.500, 0.184, 0.229, 0.587, 0.970, 0.668, 0.332, 0.000, 0.500,
       0.052, 0.880, 0.068, 0.985, 0.911, 0.085, 0.003, 1.000, 0.660,
       0.059, 0.281])

['B' 'D' 'E' 'C' 'A']


Optimization
You need to find the parameter that minimizes the -likelihood. Hint: think about potential problems associated with this optimization

In [42]:
b = (0.0,1) # indicates bounds here, since it's probability, so between 0 and 1 
beta = (0.6,1)
#bnds = (b,b,b,b,b,b,b)
bnds_row = (beta,b,b,b)
bnds = bnds_row *5


simulaiton = 1
currentmin = 0
current_solution = []
while simulaiton <=50:
    x0 = np.zeros((5,4))
    for i in range(5):
        x0[i, 0] = np.random.random()
        temp = np.random.random(3) 
        temp = temp/temp.sum()
        x0[i, 1:]=temp
    x00 = x0.flatten()
    con0 =  {'type': 'eq', 'fun': constraint0_node} 
    con1 =  {'type': 'eq', 'fun': constraint1_node} 
    con2 =  {'type': 'eq', 'fun': constraint2_node} 
    con3 =  {'type': 'eq', 'fun': constraint3_node} 
    con4 =  {'type': 'eq', 'fun': constraint4_node} 
    cons = ([con0,con1,con2,con3,con4])
    solution = minimize(likelihood_node,x00,method='SLSQP',bounds=bnds,constraints=cons)
    if currentmin > likelihood(solution.x):
        currentmin = likelihood(solution.x)
        current_solution = solution.x

    simulaiton +=1
    print(simulaiton)

<ipython-input-38-fd7a109ad1eb>:30: RuntimeWarning: divide by zero encountered in log
  ln_ps = np.log(p_s)


2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51


In [43]:
solution.x
# B : 0.600, 0.996, 0.004, 0.000,
  D : 0.965, 0.668, 0.332, 0.000,
  E : 0.600, 0.500, 0.285, 0.215, 
  C : 0.600, 1.000, 0.000, 0.000, 
  A : 1.000, 0.631, 0.069, 0.300]


array([0.600, 0.996, 0.004, 0.000, 0.965, 0.668, 0.332, 0.000, 0.600,
       0.500, 0.285, 0.215, 0.600, 1.000, 0.000, 0.000, 1.000, 0.631,
       0.069, 0.300])

### Define an error rate that is the same across all subjects

In [ ]:
def loglikelihood(epslon):
    """Assume that all subjects have the same error rate"""
    

#     decision_rules = ["RULE1","RULE2","RULE3"] (only "subject_id", epslon = .94)
#     decision_rules = ["RULE1_relaxed","RULE2","RULE3"] (only "subject_id",esplon=.95, similar)
    decision_rules = ["RULE1","RULE2","RULE3"] 
    gb = data.groupby(["subject_id"])[decision_rules].sum()
    gb_size = data.groupby(["subject_id"])[decision_rules].count()
    
    loglike = 0
    sub_likelihood = []
    for subject_id in data.subject_id.unique() :
        subject_prob = []
        
        for i in range(len(decision_rules)):
            # Total number of rounds meets the decision rule
            count = gb[decision_rules[i]].loc[subject_id]
            N = gb_size[decision_rules[i]].loc[subject_id]
           
            
            # Note: in El-Gamal and Grether (1995), the error is symmetric, thus prob of making a mistake is epslon/2
            # Since here we haven't considered the error size, just denote 1 as the waiting period meets the defined decision rule, 
            # I assume people will make an error with prob epslon
            ## LATER: define error rate for waiting too short and too long! 
            
#             prob = ((1-epslon)**(count))*(epslon**(N-count))
            log_prob = count * np.log(1-epslon) + (N-count)* np.log(epslon)
#             print([count, N, epslon])
            subject_prob.append(log_prob)
#             subject_prob.append(prob)
            
            
#         print(subject_prob) 
#         sub_likelihood.append([subject_id, subject_prob])
        sub_likelihood.append([subject_id, subject_prob])
#         print(max(subject_prob))
#         print(sub_likelihood)
        loglike  += max(subject_prob)
    
    return -loglike

In [ ]:
epslon = .9
sub_likelihood = []
for subject_id in data.subject_id.unique() :
    decision_rules = ["RULE1","RULE2","RULE3"]
    gb = data.groupby(["subject_id"])[decision_rules].sum()
    gb_size = data.groupby(["subject_id"])[decision_rules].count()
    
    subject_prob = []
    for i in range(len(decision_rules)):
        count = gb[decision_rules[i]].loc[subject_id]
        N = gb_size[decision_rules[i]].loc[subject_id]
        prob = ((1-epslon)**(count))*(epslon**(N-count))
        subject_prob.append(prob)
    
    sub_likelihood.append([subject_id, subject_prob])
# sub_likelihood[]    

### Define an error rate that is the same across different positions

In [ ]:
def loglikelihood(epslon_list):
    """Assume that all subjects have the same error rate, by node"""
    

#     decision_rules = ["RULE1","RULE2","RULE3"] (epslon = .94)
#     decision_rules = ["RULE1_relaxed","RULE2","RULE3"] (similar)
    decision_rules = ["RULE1","RULE2","RULE3"] 
    gb = data.groupby(["subject_id","p_node"])[decision_rules].sum()
    gb_size = data.groupby(["subject_id","p_node"])[decision_rules].count()
    p_node_list = data.p_node.unique()
    
    loglike = 0
    sub_likelihood = []
    
    for subject_id in data.subject_id.unique() :
        subject_prob = []
        for ii in range(5):
        
            for i in range(len(decision_rules)):
                # Total number of rounds meets the decision rule
                count = gb[decision_rules[i]].loc[subject_id,p_node_list[ii] ]
                N = gb_size[decision_rules[i]].loc[subject_id, p_node_list[ii]]
                epslon = epslon_list[ii]


                # Note: in El-Gamal and Grether (1995), the error is symmetric, thus prob of making a mistake is epslon/2
                # Since here we haven't considered the error size, just denote 1 as the waiting period meets the defined decision rule, 
                # I assume people will make an error with prob epslon
                ## LATER: define error rate for waiting too short and too long! 

    #             prob = ((1-epslon)**(count))*(epslon**(N-count))
                log_prob = count * np.log(1-epslon) + (N-count)* np.log(epslon)
    #             print([count, N, epslon])
                subject_prob.append(log_prob)
    #             subject_prob.append(prob)


    #         print(subject_prob) 
    #         sub_likelihood.append([subject_id, subject_prob])
            sub_likelihood.append([subject_id, p_node_list[ii],subject_prob])
    #         print(max(subject_prob))
    #         print(sub_likelihood)
            loglike  += max(subject_prob)
    
    return -loglike

In [ ]:
# loglikelihood([0.1,0.1,0.1,0.1,0.1])
b = (0.0,1) # indicates bounds here, since it's probability, so between 0 and 1 
beta = (0.5,1)
#bnds = (b,b,b,b,b,b,b)
bnds = (b,b,b,b,b)


res = minimize(loglikelihood,[0.1,0.1,0.1,0.1,0.1], method='SLSQP',bounds=bnds)

In [ ]:
# loglikelihood(.9)
# gb_size = data.groupby(["subject_id"])[decision_rules].count()
# gb_size
# print(sub_likelihood)

res = minimize(loglikelihood,0.5, method='SLSQP')
res
#Note: using np.log start with .1 this estimate is 0.39; start with .2, estimate is .95
#Note: 


simulation = 1
ideal_lamda = 0
max_likelihood = 20000
while simulation <=20:
#     res = minimize(loglikelihood, p0, args=(l,cost,gamma,), method='SLSQP',constraints=cons)
    lamda0  = np.random.random()
    res = minimize(loglikelihood, lamda0,  method='SLSQP')
    if loglikelihood(res.x) < max_likelihood :
        max_likelihood = loglikelihood(res.x)
        ideal_lamda = res.x
        print(ideal_lamda)
    print(simulation)
    simulation += 1 

ideal_lamda
loglike_estimate = loglikelihood(ideal_lamda)

In [ ]:
sub_likelihood

Homework. q3 Replicate Table 11 QRE MLE
Extenstion: obtain the MLE and estimated lamda for all four treatments

In [ ]:
def loglikelihood(lamda, q, t):
    '''Q is the probability of receiving correct signal, exogeneous, varies in the treatment,
    for homework 6, Q = 5/9
    In the function, capital A and B denotes actions, a and b are for signals'''
    
    # Initial p public belief as 1/2
    p = 1/2
    Q = q
    T = t
    logL = 0 
    i = 1
    # obtain signal and decision in each t , here is where my loop begins
#     signal =
#     choice = 
    for i in range(len(choiceset)):


        signal = signalset[i]
        choice = choiceset[i]

        # define agent's private posterior beliefs:
        pi_a = Q*p / (Q*p + (1-Q)*(1-p))
        pi_b = (1-Q)*p / ((1-Q)*p + Q*(1-p))

      # define probability of making choices A and B conditional on signal a and b
        P_A_a = 1 / (1 + np.exp(lamda*(1-2*pi_a)))
        P_A_b = 1/ (1 + np.exp(lamda*(1-2*pi_b)))
        P_B_a = 1 - P_A_a
        P_B_b = 1 - P_A_b

        # define the evolvement of public belief
        p_plus = (Q*p*P_A_a + p*(1-Q)*P_A_b) / ((p*Q+(1-p)*(1-Q))*P_A_a + (p*(1-Q)+(1-p)*Q)*P_A_b)
        p_minus = (p*Q*P_B_a + p*(1-Q)*P_B_b) / ((p*Q+(1-p)*(1-Q))*P_B_a + (p*(1-Q)+(1-p)*Q)*P_B_b )



        # define the log likelihood
        # Note here 0 stands for choice A and signal a, 1 is for choice B and signal b
#         if signal == 1 and choice == 1 :
#             logL += np.log(P_A_a)
#             p = p_plus
#         elif signal == 0 and choice == 1 :
#             logL += np.log(P_A_b)
#             p = p_plus
#         elif signal == 1 and choice == 0:
#             logL += np.log(P_B_a)
#             p = p_minus
#         elif signal == 0 and choice ==0 :
#             logL += np.log(P_B_b)
#             p = p_minus

#         What I thought would be the correct coding
#       Note, at the end of each match, we need to put the intial p back to 1/2
        if signal == 0 and choice == 0 :
            logL += np.log(P_A_a)
            p = p_plus
        elif signal == 1 and choice == 0 :
            logL += np.log(P_A_b)
            p = p_plus
        elif signal == 0 and choice == 1:
            logL += np.log(P_B_a)
            p = p_minus
        elif signal == 1 and choice ==1 :
            logL += np.log(P_B_b)
            p = p_minus
        
        if divmod(i ,T)[1]==0 and i != 0 :
            p = 1/2
#             print(i)
            

#     print(divmod(i ,20))

    
    return -logL
    